In [13]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

def load_stock_data(data_dir='data'):
    """
    Load all stock data
    """
    stocks = {}
    files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
    
    for file in files:
        stock = file.replace('.csv', '')
        df = pd.read_csv(os.path.join(data_dir, file), index_col=0, parse_dates=True)
        stocks[stock] = df
    
    return stocks

In [14]:
def calculate_statistics(stocks_data):
    """
    Calculate summary statistics
    """
    results = []
    
    for stock, df in stocks_data.items():
        # basic info
        start_date = df.index.min()
        end_date = df.index.max()
        num_days = len(df)
        
        # price statistics
        start_price = df['Close'].iloc[0]
        end_price = df['Close'].iloc[-1]
        min_price = df['Close'].min()
        max_price = df['Close'].max()
        avg_price = df['Close'].mean()
        median_price = df['Close'].median()
        std_price = df['Close'].std()
        
        # returns
        daily_returns = df['Close'].pct_change().dropna()
        total_return = ((end_price / start_price) - 1) * 100
        avg_daily_return = daily_returns.mean() * 100
        
        # annualized metrics
        annual_return = ((1 + daily_returns.mean())**252 - 1) * 100
        annual_volatility = daily_returns.std() * np.sqrt(252) * 100
        
        # risk metrics
        sharpe_ratio = (annual_return - 4) / annual_volatility if annual_volatility > 0 else 0  # assuming 4% risk-free rate
        
        # drawdown
        running_max = df['Close'].cummax()
        drawdown = (df['Close'] - running_max) / running_max * 100
        max_drawdown = drawdown.min()
        
        # volume statistics
        avg_volume = df['Volume'].mean()
        total_volume = df['Volume'].sum()
        
        # positive/negative days
        positive_days = (daily_returns > 0).sum()
        negative_days = (daily_returns < 0).sum()
        win_rate = (positive_days / len(daily_returns)) * 100 if len(daily_returns) > 0 else 0
        
        # best/worst days
        best_day = daily_returns.max() * 100
        worst_day = daily_returns.min() * 100
        
        results.append({
            'Stock': stock,
            'Start_Date': start_date.strftime('%Y-%m-%d'),
            'End_Date': end_date.strftime('%Y-%m-%d'),
            'Trading_Days': num_days,
            'Start_Price': round(start_price, 2),
            'End_Price': round(end_price, 2),
            'Min_Price': round(min_price, 2),
            'Max_Price': round(max_price, 2),
            'Avg_Price': round(avg_price, 2),
            'Median_Price': round(median_price, 2),
            'Std_Price': round(std_price, 2),
            'Total_Return_%': round(total_return, 2),
            'Annual_Return_%': round(annual_return, 2),
            'Annual_Volatility_%': round(annual_volatility, 2),
            'Sharpe_Ratio': round(sharpe_ratio, 3),
            'Max_Drawdown_%': round(max_drawdown, 2),
            'Avg_Volume': int(avg_volume),
            'Total_Volume': int(total_volume),
            'Positive_Days': positive_days,
            'Negative_Days': negative_days,
            'Win_Rate_%': round(win_rate, 2),
            'Best_Day_%': round(best_day, 2),
            'Worst_Day_%': round(worst_day, 2)
        })
    
    return pd.DataFrame(results)

In [15]:
def print_summary(df_stats):
    """
    Print summary
    """
    print("\n" + "="*100)
    print(" "*35 + "STOCK MARKET DATA SUMMARY")
    print("="*100 + "\n")
    
    print(f"Total Stocks Analyzed: {len(df_stats)}")
    print(f"Date Range: {df_stats['Start_Date'].min()} to {df_stats['End_Date'].max()}")
    print(f"Average Trading Days: {df_stats['Trading_Days'].mean():.0f}")
    
    print("\n" + "-"*100)
    print("TOP PERFORMERS (by Total Return)")
    print("-"*100)
    top_performers = df_stats.nlargest(10, 'Total_Return_%')[['Stock', 'Total_Return_%', 'Annual_Return_%', 'Annual_Volatility_%', 'Sharpe_Ratio']]
    print(top_performers.to_string(index=False))
    
    print("\n" + "-"*100)
    print("WORST PERFORMERS (by Total Return)")
    print("-"*100)
    worst_performers = df_stats.nsmallest(10, 'Total_Return_%')[['Stock', 'Total_Return_%', 'Annual_Return_%', 'Annual_Volatility_%', 'Sharpe_Ratio']]
    print(worst_performers.to_string(index=False))
    
    print("\n" + "-"*100)
    print("MOST VOLATILE STOCKS")
    print("-"*100)
    most_volatile = df_stats.nlargest(10, 'Annual_Volatility_%')[['Stock', 'Annual_Volatility_%', 'Total_Return_%', 'Max_Drawdown_%']]
    print(most_volatile.to_string(index=False))
    
    print("\n" + "-"*100)
    print("BEST RISK-ADJUSTED RETURNS (by Sharpe Ratio)")
    print("-"*100)
    best_sharpe = df_stats.nlargest(10, 'Sharpe_Ratio')[['Stock', 'Sharpe_Ratio', 'Annual_Return_%', 'Annual_Volatility_%']]
    print(best_sharpe.to_string(index=False))
    
    print("\n" + "-"*100)
    print("LARGEST DRAWDOWNS")
    print("-"*100)
    largest_dd = df_stats.nsmallest(10, 'Max_Drawdown_%')[['Stock', 'Max_Drawdown_%', 'Total_Return_%', 'Annual_Volatility_%']]
    print(largest_dd.to_string(index=False))
    
    print("\n" + "-"*100)
    print("HIGHEST TRADING VOLUMES")
    print("-"*100)
    highest_volume = df_stats.nlargest(10, 'Avg_Volume')[['Stock', 'Avg_Volume', 'Total_Return_%']]
    print(highest_volume.to_string(index=False))
    
    print("\n" + "-"*100)
    print("OVERALL STATISTICS")
    print("-"*100)
    print(f"Average Total Return: {df_stats['Total_Return_%'].mean():.2f}%")
    print(f"Median Total Return: {df_stats['Total_Return_%'].median():.2f}%")
    print(f"Average Annual Volatility: {df_stats['Annual_Volatility_%'].mean():.2f}%")
    print(f"Average Sharpe Ratio: {df_stats['Sharpe_Ratio'].mean():.3f}")
    print(f"Average Max Drawdown: {df_stats['Max_Drawdown_%'].mean():.2f}%")
    print(f"Average Win Rate: {df_stats['Win_Rate_%'].mean():.2f}%")
    
    print("\n" + "="*100 + "\n")

In [16]:
def sector_analysis(df_stats):
    """
    Analyze by sector
    """
    sectors = {
        'Technology': ['AAPL', 'MSFT', 'NVDA', 'GOOGL', 'META', 'AVGO', 'CSCO', 'ADBE', 'CRM', 'INTC'],
        'Financial': ['JPM', 'BAC', 'WFC', 'GS', 'MS', 'BLK', 'AXP', 'SCHW'],
        'Healthcare': ['UNH', 'JNJ', 'LLY', 'ABBV', 'MRK', 'PFE', 'TMO'],
        'Consumer': ['AMZN', 'TSLA', 'HD', 'MCD', 'NKE', 'SBUX', 'WMT', 'PG', 'KO', 'PEP'],
        'Industrial': ['BA', 'CAT', 'UPS', 'RTX', 'HON'],
        'Energy': ['XOM', 'CVX', 'COP'],
        'Communication': ['DIS', 'NFLX', 'CMCSA'],
        'Utilities': ['NEE', 'DUK'],
        'Real Estate': ['AMT', 'PLD']
    }
    
    print("\n" + "="*100)
    print(" "*35 + "SECTOR ANALYSIS")
    print("="*100 + "\n")
    
    sector_stats = []
    
    for sector, stocks in sectors.items():
        sector_df = df_stats[df_stats['Stock'].isin(stocks)]
        
        if len(sector_df) > 0:
            sector_stats.append({
                'Sector': sector,
                'Num_Stocks': len(sector_df),
                'Avg_Return_%': sector_df['Total_Return_%'].mean(),
                'Avg_Annual_Return_%': sector_df['Annual_Return_%'].mean(),
                'Avg_Volatility_%': sector_df['Annual_Volatility_%'].mean(),
                'Avg_Sharpe': sector_df['Sharpe_Ratio'].mean(),
                'Avg_Max_DD_%': sector_df['Max_Drawdown_%'].mean()
            })
    
    sector_summary = pd.DataFrame(sector_stats)
    sector_summary = sector_summary.sort_values('Avg_Return_%', ascending=False)
    
    print(sector_summary.to_string(index=False))
    print("\n" + "="*100 + "\n")
    
    return sector_summary

In [17]:
def main():
    """
    Main execution
    """
    stocks_data = load_stock_data('data')
    print(f"Loaded {len(stocks_data)} stocks\n")
    
    df_stats = calculate_statistics(stocks_data)
    
    # print summary
    print_summary(df_stats)
    
    # sector analysis
    sector_summary = sector_analysis(df_stats)
    
    # save results
    output_dir = 'eda'
    os.makedirs(output_dir, exist_ok=True)
    
    df_stats.to_csv(f'{output_dir}/summary_statistics.csv', index=False)
    sector_summary.to_csv(f'{output_dir}/sector_summary.csv', index=False)

In [18]:
if __name__ == "__main__":
    main()

Loaded 51 stocks


                                   STOCK MARKET DATA SUMMARY

Total Stocks Analyzed: 51
Date Range: 2000-01-03 to 2024-12-30
Average Trading Days: 6007

----------------------------------------------------------------------------------------------------
TOP PERFORMERS (by Total Return)
----------------------------------------------------------------------------------------------------
Stock  Total_Return_%  Annual_Return_%  Annual_Volatility_%  Sharpe_Ratio
 NVDA       153623.75            59.85                59.40         0.940
 NFLX        75159.73            56.58                55.46         0.948
 AAPL        29814.25            35.87                38.74         0.823
 TSLA        26108.24            72.92                57.38         1.201
 AVGO        20383.90            51.14                36.70         1.285
  BLK        10019.37            27.30                33.64         0.693
  UNH         9412.39            25.86                30.91         0.707
 